In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, DecimalType, TimestampType
from pyspark.sql.functions import create_map, lit, col 

In [ ]:
# META-DATA
pipeline_id = dbutils.widgets.get("pipeline_id")
run_id = dbutils.widgets.get("run_id")
task_id = dbutils.widgets.get("task_id")
processed_timestamp = dbutils.widgets.get("processed_timestamp")
#ENVIRONMENT SPECIFIC
environment = dbutils.widgets.get("environment")
region = dbutils.widgets.get("region")
catalog_name = f"citibike_{environment}"
#TABLE_NAMES
src_data_path = f'/Volumes/{catalog_name}/00_landing/source_citibike_data/JC-202503-citibike-tripdata.csv'
tgt_table_name = f"{catalog_name}.01_bronze.jc_citibike"

In [ ]:
schema = StructType([
    StructField("ride_id", StringType(), True), 
    StructField("rideable_type", StringType(), True),
    StructField("started_at", TimestampType(), True),   
    StructField("ended_at", TimestampType(), True),
    StructField("start_station_name", StringType(), True),
    StructField("start_station_id", StringType(), True),
    StructField("end_station_name", StringType(), True),
    StructField("end_station_id", StringType(), True),
    StructField("start_lat", DecimalType(10, 7), True),
    StructField("start_lng", DecimalType(10, 7), True),
    StructField("end_lat", DecimalType(10, 7), True),
    StructField("end_lng", DecimalType(10, 7), True),
    StructField("member_casual", StringType(), True)
])

df = spark.read.csv(src_data_path, header=True, schema=schema)

In [ ]:
df = df.withColumn('metadata', 
    create_map(
        lit('pipeline_id'), lit(pipeline_id),
        lit('run_id'), lit(run_id),
        lit('task_id'), lit(task_id),
        lit('processed_timestamp'), lit(processed_timestamp)
    )
)

In [ ]:
(df.write
    .mode('overwrite')
    .option("overwriteSchema","true")
    .saveAsTable(tgt_table_name)
)